### Lab Assignment: Python Warmup and Logfile Analytics 

### University of Virginia
### DS 7200: Distributed Computing
### Last Updated: August 20, 2023

---

This lab consists of two parts. Part 1 is the Python warmup. Part 2 is the logfile analytics in Spark.  
Answer the questions in this assignment, showing all code and solutions.

**Total points: 15**

---

### Part 1: Python Warmup

1) (1 PT) Rename this notebook to JupyterTutorial_[your_initials], where you will enter your initials in place of [your_initials].

2) (1 PT) In the cell below, enter a list of data science topics you find interesting.  Use the markdown style (you will need to change the style from the Code style).

- handling of missing data
- the intersection of explainability and predictability in machine learning models
- convolutional neural networks

3) (1 PT) In the cell below, enter the following Python list:

some_vals = [-1, 6, 12, 34]  

You will use the Code style, run the cell, and print the list.

In [12]:
some_vals = [-1, 6, 12, 34]
print(some_vals)

[-1, 6, 12, 34]


4) (1 PT) Use a list comprehension to return a filtered list containing only the values greater than 6.  
Call this list *some_vals_filtered* and print it.

In [2]:
some_vals_filtered = [x for x in some_vals if x > 6]
print(some_vals_filtered)

[12, 34]


Next, a small pandas dataframe is constructed.

In [3]:
import pandas as pd

df = pd.DataFrame({'first_name': ['Andy','Crystal'],
                   'domain_facebook' : [1,1],
                   'domain_foursquare' : [0,0],
                   'age' : [20, 32]})
df

,first_name,domain_facebook,domain_foursquare,age
0,Andy,1,0,20
1,Crystal,1,0,32


5) (1 PT) In the cell below, write a list comprehension that returns the fields names in the dataframe `df` containing the string *domain*.  Run the cell to verify the correct result.

In [4]:
dm = [x for x in df.columns if x.startswith("domain")]
print(dm)

['domain_facebook', 'domain_foursquare']


6) (1 PT) Use the list comprehension from (5) to index into `df` and show the data for columns containing *domain*

In [5]:
df[dm]

,domain_facebook,domain_foursquare
0,1,0
1,1,0


7) (1 PT) In the cell below, print the *domain_facebook* column

In [6]:
df[dm[0]]

0    1
1    1
Name: domain_facebook, dtype: int64

8) (1 PT) In the cell below, print the row with index 1.

In [7]:
df[1:]

,first_name,domain_facebook,domain_foursquare,age
1,Crystal,1,0,32


9) (1 PT) Next, you will cube the *age* column of `df` and assign the result to a new column called *agecube*.

Specifically, call the `apply` method with a `lambda function` inside to cube the *age* column.  
Print the dataframe.

In [8]:
df['agecube'] = df.age.apply(lambda x: x**3)
print(df)

  first_name  domain_facebook  domain_foursquare  age  agecube
0       Andy                1                  0   20     8000
1    Crystal                1                  0   32    32768


10) (1 PT) Given the list of strings below, form one string, placing semicolons between each word.  It should look like this:  

`'the;quick;brown;fox'`

Print the resulting string.

In [14]:
some_list = ['the','quick','brown','fox']

In [15]:
some_list_string = ";".join(some_list)
print(some_list_string)

the;quick;brown;fox


### Part 2: Logfile Analytics

Import modules for Spark Session and regex 

In [11]:
from pyspark.sql import SparkSession
import re

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

24/09/06 11:35:50 WARN Utils: Your hostname, Eileanors-Laptop.local resolves to a loopback address: 127.0.0.1, but we couldn't find any external IP address!
24/09/06 11:35:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/06 11:35:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


11) (1 PT) Read in the logfile.txt data

In [16]:
data_filename = 'logfile.txt'
lines = sc.textFile(data_filename)

12) (1 PT) Count the number of rows of data

In [17]:
lines.count()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py:154: DeprecationWarning: This process (pid=46276) is multi-threaded, use of fork() may lead to deadlocks in the child.


360

13) (1 PT) Show the first 5 lines

In [18]:
lines.collect()[:5]

[' 01 ',
 '03/22 08:51:01 INFO   :.main: *************** RSVP Agent started ***************',
 ' 02 ',
 '03/22 08:51:01 INFO   :...locate_configFile: Specified configuration file: /u/user10/rsvpd1.conf',
 '03/22 08:51:01 INFO   :.main: Using log level 511']

14) (1 PT) Show all lines containing the word WARNING and write code to count them

In [19]:
spark_lines = lines.filter(lambda x: "WARNING" in x)
spark_lines.collect()

['03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.',
 '03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.',
 '03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.',
 '03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.']

In [20]:
linecounts = lines.filter(lambda x: "WARNING" in x) \
            .map(lambda x: (x, 1)) \
            .reduceByKey(lambda x,y:x+y) \
            .map(lambda x:(x[1],x[0])) \
            .sortByKey(False)

linecounts.collect() #4 instances of the same WARNING

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling


[(4,
  '03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.')]

15) (1 PT) Write a word count program to count the number of each of these log levels:  

terms = 'WARNING|INFO|EVENT|PROTERR|TRACE'

In [44]:
import re
words = lines.flatMap(lambda x: re.split(' |:', x))
wordcounts = words.filter(lambda x: "WARNING" in x or "INFO" in x or "EVENT" in x or "PROTERR" in x or "TRACE" in x) \
            .map(lambda x: (x, 1)) \
            .reduceByKey(lambda x,y:x+y) \
            .map(lambda x:(x[1],x[0])) \
            .sortByKey(False)
wordcounts.collect()

[(145, 'INFO'), (119, 'TRACE'), (13, 'EVENT'), (4, 'WARNING'), (1, 'PROTERR')]